In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
import matplotlib.pyplot as plt

df = pd.read_csv("online_retail.csv")

In [2]:
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])

df["InvoiceYear"] = df["InvoiceDate"].dt.year
df["InvoiceMonth"] = df["InvoiceDate"].dt.month
df["InvoiceDay"] = df["InvoiceDate"].dt.day
df["InvoiceDow"] = df["InvoiceDate"].dt.dayofweek

In [3]:
df["Cancel"] = df["Quantity"] < 0
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,InvoiceYear,InvoiceMonth,InvoiceDay,InvoiceDow,Cancel
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,2009,12,1,1,False
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,2009,12,1,1,False
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,2009,12,1,1,False
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,2009,12,1,1,False
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,2009,12,1,1,False


In [5]:
df['InvoiceYM'] = df['InvoiceDate'].astype(str).str[:7]
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,InvoiceYear,InvoiceMonth,InvoiceDay,InvoiceDow,Cancel,InvoiceYM
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,2009,12,1,1,False,2009-12
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,2009,12,1,1,False,2009-12
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,2009,12,1,1,False,2009-12
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,2009,12,1,1,False,2009-12
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,2009,12,1,1,False,2009-12


In [6]:
df["InvoiceTime"] = df["InvoiceDate"].dt.time
df["InvoiceHour"] = df["InvoiceDate"].dt.hour
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,InvoiceYear,InvoiceMonth,InvoiceDay,InvoiceDow,Cancel,InvoiceYM,InvoiceTime,InvoiceHour
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,2009,12,1,1,False,2009-12,07:45:00,7
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,2009,12,1,1,False,2009-12,07:45:00,7
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,2009,12,1,1,False,2009-12,07:45:00,7
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,2009,12,1,1,False,2009-12,07:45:00,7
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,2009,12,1,1,False,2009-12,07:45:00,7


In [7]:
df=df[~df["Invoice"].str.contains("C", na=False)]

In [8]:
df= df[(df["Quantity"]>0)]

In [9]:
df["InvoiceDate"].max()

Timestamp('2011-12-09 12:50:00')

In [10]:
today_date = dt.datetime(2011, 12, 11)

In [12]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [13]:
df["TotalPrice"] = df["Quantity"]*df["Price"]

In [14]:
rfm = df.groupby('Customer ID').agg({
    'InvoiceDate': lambda date: (today_date - date.max()).days, 
    'Invoice': lambda num: num.nunique(),  
    'TotalPrice': lambda TotalPrice: TotalPrice.sum()  
})

In [15]:
rfm.columns = ['Recency', 'Frequency', 'Monetary']

In [16]:
#구매기록이 없는 고객 제거
rfm = rfm[(rfm["Monetary"]) > 0 & (rfm["Frequency"] > 0)]

In [17]:
rfm

,Recency,Frequency,Monetary
Customer ID,,,
12346.0,326,12,77556.46
12347.0,3,8,5633.32
12348.0,76,5,2019.40
12349.0,19,4,4428.69
12350.0,311,1,334.40
...,...,...,...
18283.0,4,22,2736.65
18284.0,432,1,461.68
18285.0,661,1,427.00


In [18]:
rfm["RecencyScore"] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1])

In [19]:
rfm["FrequencyScore"] = pd.qcut(rfm['Frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])

In [20]:
rfm["MonetaryScore"] = pd.qcut(rfm['Monetary'], 5, labels=[1, 2, 3, 4, 5])

In [21]:
rfm["RFM_SCORE"] = (rfm['RecencyScore'].astype(str) +
                    rfm['FrequencyScore'].astype(str) +
                    rfm['MonetaryScore'].astype(str))

In [22]:
seg_map = {
    r'[1-2][1-2]': 'Hibernating',
    r'[1-2][3-4]': 'At_Risk',
    r'[1-2]5': 'Cant_Loose',
    r'3[1-2]': 'About_to_Sleep',
    r'33': 'Need_Attention',
    r'[3-4][4-5]': 'Loyal_Customers',
    r'41': 'Promising',
    r'51': 'New_Customers',
    r'[4-5][2-3]': 'Potential_Loyalists',
    r'5[4-5]': 'Champions'
}

In [23]:
rfm['Segment'] = rfm['RecencyScore'].astype(str) + rfm['FrequencyScore'].astype(str)

In [25]:
rfm['Segment'] = rfm['Segment'].replace(seg_map, regex=True)
rfm

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE,Segment
Customer ID,,,,,,,,
12346.0,326,12,77556.46,2,5,5,255,Cant_Loose
12347.0,3,8,5633.32,5,4,5,545,Champions
12348.0,76,5,2019.40,3,4,4,344,Loyal_Customers
12349.0,19,4,4428.69,5,3,5,535,Potential_Loyalists
12350.0,311,1,334.40,2,1,2,212,Hibernating
...,...,...,...,...,...,...,...,...
18283.0,4,22,2736.65,5,5,4,554,Champions
18284.0,432,1,461.68,1,2,2,122,Hibernating
18285.0,661,1,427.00,1,2,2,122,Hibernating


In [26]:
rfm[["Segment", "Recency", "Frequency", "Monetary"]].groupby("Segment").agg(["mean", "count"])
rfm

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE,Segment
Customer ID,,,,,,,,
12346.0,326,12,77556.46,2,5,5,255,Cant_Loose
12347.0,3,8,5633.32,5,4,5,545,Champions
12348.0,76,5,2019.40,3,4,4,344,Loyal_Customers
12349.0,19,4,4428.69,5,3,5,535,Potential_Loyalists
12350.0,311,1,334.40,2,1,2,212,Hibernating
...,...,...,...,...,...,...,...,...
18283.0,4,22,2736.65,5,5,4,554,Champions
18284.0,432,1,461.68,1,2,2,122,Hibernating
18285.0,661,1,427.00,1,2,2,122,Hibernating


In [27]:
rfm[["Segment", "Recency", "Frequency", "Monetary"]].groupby("Segment").agg(["mean", "count","min", "median", "max"])

Recency                         Frequency            \
                           mean count  min median  max       mean count min   
Segment                                                                       
About_to_Sleep       107.618557   388   60   94.0  190   1.360825   388   1   
At_Risk              373.942667   750  191  377.0  734   3.904000   750   2   
Cant_Loose           331.802817    71  193  326.0  577  15.929577    71   8   
Champions              9.250000   852    1    9.0   20  19.217136   852   4   
Hibernating          460.012484  1522  191  435.0  739   1.252300  1522   1   
Loyal_Customers       68.306016  1147   22   54.0  190   9.803836  1147   4   
Need_Attention       114.241636   269   60  107.0  190   3.156134   269   2   
New_Customers         11.303571    56    1   11.0   20   1.000000    56   1   
Potential_Loyalists   26.297335   713    1   24.0   59   2.589060   713   1   
Promising             39.281818   110   22   38.5   59   1.000000   110   1   

                                     Monetary                          \
                    median  max          mean count     min    median   
Segment                                                                 
About_to_Sleep         1.0    2    531.969905   388    6.20   370.175   
At_Risk                4.0    8   1383.596253   750   38.92   965.510   
Cant_Loose            11.0  155   8355.677634    71  628.32  3881.890   
Champions             11.5  398  10795.520581   852  357.00  4009.870   
Hibernating            1.0    2    437.963135  1522    2.95   285.205   
Loyal_Customers        8.0  107   4199.728592  1147   30.95  2602.320   
Need_Attention         3.0    4   1283.114015   269  120.90   977.170   
New_Customers          1.0    1    356.257857    56   43.20   262.680   
Potential_Loyalists    3.0    4   1155.427281   713   20.80   692.190   
Promising              1.0    1    324.497000   110   30.00   221.620   

                                
                           max  
Segment                         
About_to_Sleep         5416.32  
At_Risk               44534.30  
Cant_Loose            77556.46  
Champions            608821.65  
Hibernating           13916.34  
Loyal_Customers      144458.37  
Need_Attention        39916.50  
New_Customers          3861.00  
Potential_Loyalists  168472.50  
Promising              1635.66